In [7]:
import pandas as pd

# 소비자 구매 데이터 (예시)
purchase_data = pd.DataFrame({
    'customer_id': [1, 1, 2, 2, 3, 3],
    'item': ['쌀', '배추', '쌀', '고춧가루', '배추', '무']
})

# 상품 속성 데이터
item_data = pd.DataFrame({
    'item': ['쌀', '배추', '고춧가루', '무', '천일염'],
    'tags': ['곡물 건강', '채소 김장 필수', '조미료 매운맛', '채소 김치', '조미료 소금']
})

콘텐츠 기반 추천


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 상품 태그 데이터를 TF-IDF로 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(item_data['tags'])

# 아이템 간 유사도 계산
content_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 결과를 데이터프레임으로 변환
content_similarity_df = pd.DataFrame(content_similarity, index=item_data['item'], columns=item_data['item'])
print("콘텐츠 기반 유사도:\n", content_similarity_df)


콘텐츠 기반 유사도:
 item    쌀        배추      고춧가루         무       천일염
item                                             
쌀     1.0  0.000000  0.000000  0.000000  0.000000
배추    0.0  1.000000  0.000000  0.311146  0.000000
고춧가루  0.0  0.000000  1.000000  0.000000  0.394276
무     0.0  0.311146  0.000000  1.000000  0.000000
천일염   0.0  0.000000  0.394276  0.000000  1.000000


협업 필터링

In [3]:
# 사용자-아이템 매트릭스 생성
user_item_matrix = purchase_data.pivot_table(index='customer_id', columns='item', aggfunc=lambda x: 1, fill_value=0)

# 아이템 간 유사도 계산
collab_similarity = cosine_similarity(user_item_matrix.T)
collab_similarity_df = pd.DataFrame(collab_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)
print("협업 필터링 유사도:\n", collab_similarity_df)


협업 필터링 유사도:
 item      고춧가루         무        배추         쌀
item                                        
고춧가루  1.000000  0.000000  0.000000  0.707107
무     0.000000  1.000000  0.707107  0.000000
배추    0.000000  0.707107  1.000000  0.500000
쌀     0.707107  0.000000  0.500000  1.000000


In [6]:
# 가중치 설정
content_weight = 0.6
collab_weight = 0.4

# 하이브리드 유사도 계산
hybrid_similarity = (content_weight * content_similarity_df) + (collab_weight * collab_similarity_df)
print(hybrid_similarity)


item      고춧가루        무       배추         쌀  천일염
item                                           
고춧가루  1.000000  0.00000  0.00000  0.282843  NaN
무     0.000000  1.00000  0.46953  0.000000  NaN
배추    0.000000  0.46953  1.00000  0.200000  NaN
쌀     0.282843  0.00000  0.20000  1.000000  NaN
천일염        NaN      NaN      NaN       NaN  NaN


In [5]:
def recommend_items(item, hybrid_similarity_matrix, top_n=3):
    if item not in hybrid_similarity_matrix.columns:
        return []
    # 유사도 정렬 후 상위 N개 품목 반환
    recommendations = hybrid_similarity_matrix[item].sort_values(ascending=False).index[1:top_n+1].tolist()
    return recommendations

# '배추'를 구매한 소비자에게 추천
recommendations = recommend_items('배추', hybrid_similarity)
print(f"'배추'와 관련된 추천 품목: {recommendations}")


'배추'와 관련된 추천 품목: ['무', '쌀', '고춧가루']
